In [6]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 1.05835 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mol.build()

mf = scf.RHF(mol)#.density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Oct  7 20:18:49 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpg9oqsnyp
max_memory 4000 MB (current use 328 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -0.748858511004994
  HOMO = -0.426161068043635  LUMO = 0.0766877521797008
cycle= 1 E= -1.08857074742192  delta_E= -0.34  |g|= 0.043  |ddm|= 0.654
  HOMO = -0.517282475779981  LUMO = 0.12692550158159
cycle= 2 E= -1.08926784480653  delta_E= -0.000697  |g|= 0.00615  |ddm|= 0.0304
  HOMO = -0.512615488754335  LUMO = 0.130822131736062
cycle= 3 E= -1.08928341525217  delta_E= -1.56e-05  |g|= 0.000131  |ddm|= 0.00573
  HOMO = -0.51252067419782  LUMO = 0.130887105448923
cycle= 4 E= -1.0892834232258  delta_E= -7.97e-09  |g

In [ ]:
nH = np.array([2,10,20,30])
e_pt = np.array([-1.110291,-5.700059,-12.043936,-19.424740])

-1.1400118000000001

In [7]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 2,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [8]:
from ad_afqmc import pyscf_interface, mpi_jax, wavefunctions
from ad_afqmc.cisd_perturb import sample_pt
pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

# sample_pt.run_afqmc_cisd_pt(options)

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (1, 1)
# Number of basis functions: 10
# Number of Cholesky vectors: 53
#
# Number of MPI ranks: 1
#
# norb: 10
# nelec: (1, 1)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 2
# seed: 2
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [31]:
ept = np.array([-1.109868,-2.230801,-3.363547,-4.506602])
eog = np.array([-1.108873,-2.217463,-3.325365,-4.432684])
nH = np.array([1,2,3,4])
print(ept/nH)
print(eog/nH)

[-1.109868   -1.1154005  -1.12118233 -1.1266505 ]
[-1.108873  -1.1087315 -1.108455  -1.108171 ]


In [ ]:
# H2
# AFQMC/CISD_PT energy: -1.109868 +/- 0.000154
# AFQMC/CISD_OG energy: -1.108873 +/- 0.000000
# H4
# AFQMC/CISD_PT energy: -2.230801 +/- 0.001449
# AFQMC/CISD_OG energy: -2.217463 +/- 0.000114
# H6
# AFQMC/CISD_PT energy: -3.363547 +/- 0.004001
# AFQMC/CISD_OG energy: -3.325365 +/- 0.000157
# H8
# AFQMC/CISD_PT energy: -4.506602 +/- 0.006388
# AFQMC/CISD_OG energy: -4.432684 +/- 0.000155

In [ ]:
##############################################
# H2 
# perturbation order: 2
# AFQMC/CISD_PT energy: -1.061530 +/- 0.000197
# AFQMC/CISD_OG energy: -1.061535 +/- 0.000000
# perturbation order: 4
# AFQMC/CISD_PT energy: -1.061534 +/- 0.000005
# AFQMC/CISD_OG energy: -1.061535 +/- 0.000000
##############################################
# H4
# perturbation order: 2
# AFQMC/CISD_PT energy: -2.067483 +/- 0.012023
# AFQMC/CISD_OG energy: -2.011336 +/- 0.002036
# perturbation order: 4
# AFQMC/CISD_PT energy: -2.012654 +/- 0.002468
# AFQMC/CISD_OG energy: -2.011239 +/- 0.002183
##############################################

In [ ]:
# trial_pt = wavefunctions.cisd_pt(trial.norb, trial.nelec,n_batch=trial.n_batch)
# _calc_ept = trial_pt._calc_energy_pt_restricted
# calc_ept = trial_pt.calc_energy_pt_restricted

In [9]:
from mpi4py import MPI
from jax import random

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

# trial_rdm1 = trial.get_rdm1(wave_data)
# if "rdm1" not in wave_data:
#     wave_data["rdm1"] = trial_rdm1

trial_rhf = wavefunctions.rhf(trial.norb,trial.nelec)
nocc = trial.nelec[0]
wave_data['mo_coeff'] = np.eye(trial.norb)[:,:nocc]
ham_data = trial_rhf._build_measurement_intermediates(ham_data,wave_data)
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)

# v0 the one-body energy from the reordering of the 
# two-body operators into non-normal ordered form
norb = trial.norb
chol = ham_data["chol"].reshape(-1, norb, norb)
h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
v0 = 0.5 * jnp.einsum("gik,gjk->ij",
                        chol.reshape(-1, norb, norb),
                        chol.reshape(-1, norb, norb),
                        optimize="optimal")
h1_mod = h1 - v0 
ham_data['h1_mod'] = h1_mod

init_walkers = None
prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
prop_data['n_killed_walkers'] = 0
prop_data["key"] = random.PRNGKey(seed + rank)
init_walker = prop_data['walkers'][0]
print(prop_data['e_estimate'])
print(mycc.e_tot)
print(trial_rhf._calc_energy_restricted(init_walker,ham_data,wave_data))
print(mf.e_tot)

-1.1306879221783035
-1.1306879154559715
(-1.089283428571869+0j)
-1.0892834232258277


In [10]:
from jax import lax
from jax import config, jit
config.update("jax_enable_x64", True)

@jax.jit
def _cisd_walker_olp(walker,wave_data):
    ci1,ci2 = wave_data['ci1'], wave_data['ci2']
    nocc = walker.shape[1]
    GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    o1 = jnp.einsum("ia,ia", ci1, GF[:, nocc:])
    o2 = 2 * jnp.einsum(
        "iajb, ia, jb", ci2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", ci2, GF[:, nocc:], GF[:, nocc:])
    return (1.0 + 2 * o1 + o2) * o0

@jax.jit
def _cisd_olp_exp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    <psi_ccsd|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    # olp = trial._calc_overlap_restricted(walker_1x, wave_data)
    olp = _cisd_walker_olp(walker_1x, wave_data)
    return olp

@jax.jit
def _cisd_olp_exp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    <psi_ccsd|exp(x*h2_mod)|walker>/<psi_ccsd|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _cisd_walker_olp(walker_2x, wave_data)
    # olp = trial._calc_overlap_restricted(walker_2x, wave_data)
    return olp

@partial(jit, static_argnums=(3,))
def _cisd_walker_energy(
    walker: jax.Array,
    ham_data: dict,
    wave_data: dict,
    trial: wavefunctions,
    eps :float = 3e-4
):

    norb = trial.norb
    chol = ham_data["chol"].reshape(-1, norb, norb)
    h1_mod = ham_data['h1_mod']
    # h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    # # v1 the one-body energy from the reordering of the 
    # # two-body operators into non-normal ordered form
    # v0 = 0.5 * jnp.einsum("gik,gjk->ij",
    #                         chol.reshape(-1, norb, norb),
    #                         chol.reshape(-1, norb, norb),
    #                         optimize="optimal")
    # h1_mod = h1 - v0 
    # cisd_olp = trial._calc_overlap_restricted(walker, wave_data)
    # cisd_olp = _cisd_walker_olp(walker,wave_data)
    # zero body
    h0 = ham_data["h0"]
    # mod_olp = _frg_mod_ci_olp_restricted(walker,wave_data)

    x = 0.0
    # one body
    f1 = lambda a: _cisd_olp_exp1(a,h1_mod,walker,wave_data)
    overlap, d_overlap = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _cisd_olp_exp2(eps,c,walker,wave_data)

    _, overlap_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, overlap_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, overlap_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d_2_overlap = (overlap_p - 2.0 * overlap_0 + overlap_m) / eps / eps
    
    e_cisd = h0 + (d_overlap + jnp.sum(d_2_overlap) / 2.0 )/ overlap

    return e_cisd

In [11]:
@jax.jit
def _hf_walker_olp(walker):
    nocc = walker.shape[1]
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    return o0

@jax.jit
def _hf_olp_exp1(x: float, h1_mod: jax.Array, walker: jax.Array) -> complex:
    '''
    <psi_0|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    # olp = trial._calc_overlap_restricted(walker_1x, wave_data)
    olp = _hf_walker_olp(walker_1x)
    return olp

@jax.jit
def _hf_olp_exp2(x: float, chol_i: jax.Array, walker: jax.Array) -> complex:
    '''
    <psi_0|exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _hf_walker_olp(walker_2x)
    # olp = trial._calc_overlap_restricted(walker_2x, wave_data)
    return olp

def _e0(walker, ham_data, eps=3e-4):
    norb = trial.norb
    h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    x = 0.0
    f1 = lambda a: _hf_olp_exp1(a,h1_mod,walker)
    olp, d_olp = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps, walker = carry
        return carry, _hf_olp_exp2(eps,c,walker)

    _, olp_p = lax.scan(scanned_fun, (eps, walker), chol)
    _, olp_0 = lax.scan(scanned_fun, (0.0, walker), chol)
    _, olp_m = lax.scan(scanned_fun, (-1.0 * eps, walker), chol)
    d_2_olp = (olp_p - 2.0 * olp_0 + olp_m) / eps / eps

    e_0= h0 + (d_olp + jnp.sum(d_2_olp) / 2.0 )/ olp

    return e_0

In [12]:
print(_e0(init_walker,ham_data))
print(mf.e_tot)

(-1.0892834338083683+0j)
-1.0892834232258277


In [13]:
@jax.jit
def _ci1_walker_olp(walker: jax.Array,wave_data: dict):
    '''ci_ia <psi_i^a|phi>'''
    ci1= wave_data['ci1']
    nocc = walker.shape[1]
    GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    o1 = jnp.einsum("ia,ia", ci1, GF[:, nocc:])
    return 2 * o0 * o1

@jax.jit
def _ci1_olp_exp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    olp = _ci1_walker_olp(walker_1x,wave_data)
    return olp

@jax.jit
def _ci1_olp_exp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _ci1_walker_olp(walker_2x,wave_data)
    return olp

def _e1(walker,ham_data,wave_data,eps=3e-4):
    norb = trial.norb
    h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    hf_olp = _hf_walker_olp(walker)
    ci1_olp = _ci1_walker_olp(walker,wave_data)

    # one body
    x = 0.0
    f1 = lambda a: _ci1_olp_exp1(a,h1_mod,walker,wave_data)
    _, d_olp = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _ci1_olp_exp2(eps,c,walker,wave_data)

    _, olp_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, olp_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, olp_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d_2_olp = (olp_p - 2.0 * olp_0 + olp_m) / eps / eps

    e_1 = (h0 * ci1_olp + d_olp + jnp.sum(d_2_olp) / 2.0 ) / hf_olp

    return e_1

In [14]:
_e1(init_walker,ham_data,wave_data)

Array(-3.35382682e-10+0.j, dtype=complex128)

In [15]:
@jax.jit
def _ci2_walker_olp(walker,wave_data):
    ''' c_ij^ab <psi_ij^ab|phi> '''
    ci2 = wave_data['ci2']
    nocc = walker.shape[1]
    GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    o2 = 2 * jnp.einsum(
        "iajb, ia, jb", ci2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", ci2, GF[:, nocc:], GF[:, nocc:])
    return o2 * o0

@jax.jit
def _ci2_olp_exp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    olp = _ci2_walker_olp(walker_1x,wave_data)
    return olp

@jax.jit
def _ci2_olp_exp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _ci2_walker_olp(walker_2x,wave_data)
    return olp

def _e2(walker,ham_data,wave_data,eps = 3e-4):
    norb = trial.norb
    h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

   
    # zero body
    # c_ij^ab <psi_ij^ab|phi>/<psi_0|phi> * h0
    hf_olp = _hf_walker_olp(walker)
    ci2_olp = _ci2_walker_olp(walker,wave_data)

    # one body
    # c_ij^ab <psi_ij^ab|phi_1x>/<psi_0|phi>
    x = 0.0
    f1 = lambda a: _ci2_olp_exp1(a,h1_mod,walker,wave_data)
    _, d_olp = jvp(f1, [x], [1.0])

    # two body
    # c_ij^ab <psi_ij^ab|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _ci2_olp_exp2(eps,c,walker,wave_data)

    _, olp_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, olp_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, olp_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d_2_olp = (olp_p - 2.0 * olp_0 + olp_m) / eps / eps
    
   
    e_2 = (h0 * ci2_olp + d_olp + jnp.sum(d_2_olp) / 2.0 ) / hf_olp

    return e_2

In [20]:
def _cisd_walker_energy2(walker,ham_data,wave_data):
    ci1,ci2 = wave_data['ci1'],wave_data['ci2']
    nocc = trial.nelec[0]
    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green = green[:nocc,:]
    green_occ = green[:, nocc:].copy()
    ci1g = jnp.einsum("ia,ia->", ci1, green_occ, optimize="optimal")
    ci2g_c = jnp.einsum("iajb,ia->jb", ci2, green_occ)
    ci2g_e = jnp.einsum("iajb,ib->ja", ci2, green_occ)
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = jnp.einsum("ia,ia->", ci2g, green_occ, optimize="optimal")
    # olp = 1 + 2*ci1g + gci2g
    c1 = 2*ci1g
    c2 = gci2g
    e0 = _e0(walker,ham_data)
    e1 = _e1(walker,ham_data,wave_data)
    e2 = _e2(walker,ham_data,wave_data)
    E0 = e0
    E1 = e1 - c1*e0
    E2 = e2 - c1*e1 + (c1**2-c2)*e0
    e_pt = jnp.real(E0+E1+E2)
    e_og = jnp.real((e0+e1+e2)/(1+c1+c2))
    return e_pt,e_og

In [17]:
trial._calc_energy_restricted(init_walker, ham_data, wave_data)

Array(-1.13068792+0.j, dtype=complex128)

In [18]:
trial_rhf._calc_energy_restricted(prop_data['walkers'][0],ham_data,wave_data)

Array(-1.08928343+0.j, dtype=complex128)

In [21]:
e_cisd1 = _cisd_walker_energy(prop_data['walkers'][0],ham_data,wave_data,trial)
e_cisd2 = _cisd_walker_energy2(prop_data['walkers'][0],ham_data,wave_data)
print(e_cisd1)
print(e_cisd2)

(-1.1306879352869244+0j)
(Array(-1.13068793, dtype=float64), Array(-1.13068793, dtype=float64))


In [29]:
prop_data, (block_energy, block_weight) \
    = sampler._block_scan(prop_data,None,ham_data,prop,trial,wave_data)
e_cisd = trial._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)
e_cisd1 = _cisd_walker_energy(prop_data['walkers'][0],ham_data,wave_data,trial)
e_cisd_pt, e_cisd_og = _cisd_walker_energy2(prop_data['walkers'][0],ham_data,wave_data)
print(e_cisd)
print(e_cisd1)
print(e_cisd_og)
print(e_cisd_pt)

(-1.1306879334828799-3.1513106571459137e-10j)
(-1.1306879307678406+2.089750929672679e-08j)
-1.130687964526556
-1.1308625239322474


In [6]:
e_cisd = _cisd_walker_energy(init_walker,ham_data,wave_data,trial)
print(e_cisd)
print(prop_data['e_estimate'])
print(mycc.e_tot)
print(mycc.e_tot-e_cisd)
print(prop_data['e_estimate']-e_cisd)

(-3.1696890335603705+0j)
-3.169688859436266
-3.1696889262316006
(1.0732876987162854e-07+0j)
(1.7412410446837612e-07+0j)


In [ ]:
# def thouless_trans(t1):
#     q, r = np.linalg.qr(t1)
#     u_ai = r.T
#     u_ji = q
#     u_occ = np.vstack((u_ji,u_ai))
#     u, _, _ = np.linalg.svd(u_occ)
#     return u

In [ ]:
# t1 = wave_data["ci1"]
# nocc = trial.nelec[0]
# u_t1 = thouless_trans(t1)
# u_occ = u_t1[:,:nocc]
# wave_data['u_t1'] = u_occ

In [ ]:
# from ad_afqmc import wavefunctions
# calc_e_rhf = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                                 )._calc_energy_restricted
# calc_e_rhf()

In [ ]:
# walker = prop_data['walkers'][0]
# green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
# # green = (walker.dot(jnp.linalg.inv(u_occ.T @ walker))) @ u_occ.T
# print(green.shape)
# print(green[:nocc,nocc:])

(3, 6)
[[ 1.39059730e-17+0.j  1.43713517e-16+0.j -3.71730138e-16+0.j]
 [-9.02056208e-17+0.j -1.66533454e-16+0.j  5.31171182e-33+0.j]
 [-4.74822782e-17+0.j -1.59134048e-16+0.j  4.91004945e-17+0.j]]


In [ ]:
# ci1 = wave_data["ci1"]
# g_ia = green[:nocc,nocc:]
# ci1g = jnp.einsum("ia,ia->", ci1, g_ia, optimize="optimal")
# print(ci1g)

(6.539875472771294e-19+0j)


In [ ]:
# ci2g_c = jnp.einsum("iajb,ia->jb", ci2, g_ia)
# ci2g_e = jnp.einsum("iajb,ib->ja", ci2, g_ia)
# ci2g = 2 * ci2g_c - ci2g_e
# gci2g = jnp.einsum("ia,ia->", ci2g, g_ia,optimize="optimal")
# print(gci2g)

(-2.846813597073128e-32+0j)


In [ ]:
ci2g = 2 * ci2g_c - ci2g_e
ci2g_c = jnp.einsum("iajb,ia->jb", ci2, green_occ)
ci2g_e = jnp.einsum("iajb,ib->ja", ci2, green_occ)
ci1g = jnp.einsum("ia,ia->", ci1, green_occ, optimize="optimal")
gci2g = jnp.einsum("ia,ia->", ci2g, green_occ, optimize="optimal")
olp = 1 + 2*ci1g - gci2g

In [ ]:
from jax import lax
def _calc_ccsd_walker_pert_energy(trial, walker, ham_data, wave_data):
    ci1, ci2, u_t1 = wave_data["ci1"], wave_data["ci2"], wave_data['u_t1']
    ci2 = ci2 - jnp.einsum('ia,jb->iajb',ci1,ci1)
    ci1 = np.zeros_like(ci1)
    # ci2 = np.zeros_like(ci2)
    nocc = trial.nelec[0]
    green = (walker.dot(jnp.linalg.inv(u_t1.T @ walker))) @ u_t1.T
    green = green[:nocc,:]
    green_occ = green[:, nocc:].copy()
    greenp = jnp.vstack((green_occ, -jnp.eye(trial.norb - nocc)))

    chol = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    rot_chol = chol[:, : trial.nelec[0], :]
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = jnp.einsum("pj,pj->", h1[:nocc, :], green)

    # 0 body energy
    e0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # single excitations
    # ci1g = jnp.einsum("ia,ia->", ci1, green_occ, optimize="optimal")
    # e1_1_1 = 4 * ci1g * hg
    # gpci1 = greenp @ ci1.T
    # ci1_green = gpci1 @ green
    # e1_1_2 = -2 * jnp.einsum("ij,ij->", h1, ci1_green, optimize="optimal")
    # e1_1 = e1_1_1 + e1_1_2

    # double excitations
    # ci2g_c = jnp.einsum("ptqu,pt->qu", ci2, green_occ)
    ci2g_c = jnp.einsum("iajb,ia->jb", ci2, green_occ)
    ci2g_e = jnp.einsum("iajb,ib->ja", ci2, green_occ)
    ci2_green_c = (greenp @ ci2g_c.T) @ green
    ci2_green_e = (greenp @ ci2g_e.T) @ green
    ci2_green = 2 * ci2_green_c - ci2_green_e
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = jnp.einsum("ia,ia->", ci2g, green_occ, optimize="optimal")
    e1_2_1 = 2 * hg * gci2g
    e1_2_2 = -2 * jnp.einsum("ij,ij->", h1, ci2_green, optimize="optimal")
    e1_2 = e1_2_1 + e1_2_2
    # e1 = e1_0 + e1_1 + e1_2
    e1 = e1_0 + e1_2

    # two body energy
    # ref
    lg = jnp.einsum("gpj,pj->g", rot_chol, green, optimize="optimal")
    # lg1 = jnp.einsum("gpj,pk->gjk", rot_chol, green, optimize="optimal")
    lg1 = jnp.einsum("gpj,qj->gpq", rot_chol, green, optimize="optimal")
    e2_0_1 = 2 * lg @ lg
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    # e2_1_1 = 2 * e2_0 * ci1g
    # lci1g = jnp.einsum("gij,ij->g", chol, ci1_green, optimize="optimal")
    # e2_1_2 = -2 * (lci1g @ lg)
    # lci1g1 = jnp.einsum("gij,jk->gik", chol, ci1_green, optimize="optimal")
    # glgpci1 = jnp.einsum(("gpi,iq->gpq"), gl, gpci1, optimize="optimal")
    # ci1g1 = ci1 @ green_occ.T
    # e2_1_3 = jnp.einsum("gpq,gpq->", glgpci1, lg1, optimize="optimal")
    # e2_1_3_1 = jnp.einsum("gpq,gqr,rp->", lg1, lg1, ci1g1, optimize="optimal")
    # lci1g = jnp.einsum("gip,qi->gpq", ham_data["lci1"], green, optimize="optimal")
    # e2_1_3_2 = -jnp.einsum("gpq,gqp->", lci1g, lg1, optimize="optimal")
    # e2_1_3 = e2_1_3_1 + e2_1_3_2
    # e2_1 = e2_1_1 + 2 * (e2_1_2 + e2_1_3)

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g = jnp.einsum("gij,ij->g", chol, ci2_green, optimize="optimal")
    e2_2_2_1 = -lci2g @ lg

    # lci2g1 = jnp.einsum("gij,jk->gik", chol, ci2_green, optimize="optimal")
    # lci2_green = jnp.einsum("gpi,ji->gpj", rot_chol, ci2_green, optimize="optimal")
    # e2_2_2_2 = 0.5 * jnp.einsum("gpi,gpi->", gl, lci2_green, optimize="optimal")
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = jnp.einsum("pj,ji->pi", green, chol_i, optimize="optimal")
        lci2_green_i = jnp.einsum(
            "pi,ji->pj", rot_chol_i, ci2_green, optimize="optimal"
        )
        carry[0] += 0.5 * jnp.einsum(
            "pi,pi->", gl_i, lci2_green_i, optimize="optimal"
        )
        glgp_i = jnp.einsum("pi,it->pt", gl_i, greenp, optimize="optimal").astype(
            jnp.complex64
        )
        l2ci2_1 = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_2 = jnp.einsum(
            "pu,qt,ptqu->",
            glgp_i,
            glgp_i,
            ci2.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += 2 * l2ci2_1 - l2ci2_2
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    # e2 = e2_0 + e2_1 + e2_2
    e2 = e2_0 + e2_2

    # overlap
    # overlap_1 = 2 * ci1g  # jnp.einsum("ia,ia", ci1, green_occ)
    overlap_2 = gci2g
    # overlap = 1.0 + overlap_1 + overlap_2
    overlap = 1.0 + overlap_2
    # olp_inv = 1 - 2*ci1g - gci2g + (2*ci1g)**2
    olp_inv = 1 - gci2g + gci2g**2
    
    e_pert = (e1 + e2) * olp_inv + e0
    e_orig = (e1 + e2) / overlap + e0

    # return (e1 + e2) / overlap + e0
    return e_pert, e_orig, gci2g

In [ ]:
prop_data, (block_energy, block_weight) \
    = sampler._block_scan(prop_data,None,ham_data,prop,trial,wave_data)
e_cisd = trial._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)
e_cisd_ad = _cisd_walker_energy(prop_data['walkers'][0],ham_data,wave_data,trial)
print(e_cisd)
print(e_cisd_ad)
print(e_cisd_ad-e_cisd)

In [ ]:
# e_pt, e_og = calc_ept(prop_data['walkers'],ham_data,wave_data)
# print(e_pt)
# print(e_og)

[-2.11122414+0.00012413j -2.11035962-0.00060481j]
[-2.11161642+0.00052951j -2.11010058-0.00108187j]


In [20]:
e_cisd = trial._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)
e_cisd_ad = _cisd_walker_energy(prop_data['walkers'][0],ham_data,wave_data,trial)
print(e_cisd)
print(e_cisd_ad)
print(e_cisd_ad-e_cisd)

(-3.1670389076467966-0.0010301384697907466j)
(-3.167037763930943-0.0010299900618797834j)
(1.143715853579863e-06+1.4840791096319868e-07j)


In [8]:
trial._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)

Array(-3.16935951-0.00071635j, dtype=complex128)

In [ ]:
# t2_0 = np.zeros(mycc.t2.shape)
# eris = mycc.ao2mo(mycc.mo_coeff)
# eccsd = mycc.energy(mycc.t1, t2_0, eris)
# ecc_t2_0 = eccsd+mf.e_tot
# print(ecc_t2_0,eccsd)

-2.7501475540010003 2.4901828913079822e-06
